In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/creditcard.csv')

In [ ]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
df['log10_amount'] = np.log10(df.Amount + 0.00001)

In [ ]:
RANDOM_SEED = 42
TRAINING_SAMPLE = 200000
VALIDATE_SIZE = 0.2

In [ ]:
df = df.drop(['Time','Amount'], axis=1)

# splitting by class
fraud = df[df.Class == 1]
clean = df[df.Class == 0]

In [ ]:
clean = clean.sample(frac=1).reset_index(drop=True)

# training set: exlusively non-fraud transactions
X_train = clean.iloc[:TRAINING_SAMPLE].drop('Class', axis=1)

# testing  set: the remaining non-fraud + all the fraud
X_test = clean.iloc[TRAINING_SAMPLE:].append(fraud).sample(frac=1)

<ipython-input-5-7cb7bd725fff>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X_test = clean.iloc[TRAINING_SAMPLE:].append(fraud).sample(frac=1)


In [ ]:
from sklearn.model_selection import train_test_split

# train // validate - no labels since they're all clean anyway
X_train, X_validate = train_test_split(X_train,
                                       test_size=VALIDATE_SIZE,
                                       random_state=RANDOM_SEED)

# manually splitting the labels from the test df
X_test, y_test = X_test.drop('Class', axis=1).values, X_test.Class.values

In [ ]:
from sklearn.preprocessing import Normalizer, MinMaxScaler
from sklearn.pipeline import Pipeline

# configure our pipeline
pipeline = Pipeline([('normalizer', Normalizer()),
                     ('scaler', MinMaxScaler())])

In [ ]:
pipeline.fit(X_train)

Pipeline(steps=[('normalizer', Normalizer()), ('scaler', MinMaxScaler())])

In [ ]:
X_train_transformed = pipeline.transform(X_train)
X_validate_transformed = pipeline.transform(X_validate)

In [ ]:
input_dim = X_train_transformed.shape[1]
BATCH_SIZE = 256
EPOCHS = 100

# https://keras.io/layers/core/
autoencoder = tf.keras.models.Sequential([

    # deconstruct / encode
    tf.keras.layers.Dense(input_dim, activation='elu', input_shape=(input_dim, )),
    tf.keras.layers.Dense(16, activation='elu'),
    tf.keras.layers.Dense(8, activation='elu'),
    tf.keras.layers.Dense(4, activation='elu'),
    tf.keras.layers.Dense(2, activation='elu'),

    # reconstruction / decode
    tf.keras.layers.Dense(4, activation='elu'),
    tf.keras.layers.Dense(8, activation='elu'),
    tf.keras.layers.Dense(16, activation='elu'),
    tf.keras.layers.Dense(input_dim, activation='elu')

])

# https://keras.io/api/models/model_training_apis/
autoencoder.compile(optimizer="adam",
                    loss="mse",
                    metrics=["acc"])

# print an overview of our model
autoencoder.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 29)                870       
                                                                 
 dense_10 (Dense)            (None, 16)                480       
                                                                 
 dense_11 (Dense)            (None, 8)                 136       
                                                                 
 dense_12 (Dense)            (None, 4)                 36        
                                                                 
 dense_13 (Dense)            (None, 2)                 10        
                                                                 
 dense_14 (Dense)            (None, 4)                 12        
                                                                 
 dense_15 (Dense)            (None, 8)                

In [ ]:
history = autoencoder.fit(
    X_train_transformed, X_train_transformed,
    shuffle=True,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_validate_transformed, X_validate_transformed)
)

Epoch 1/100
625/625 [==============================] - 10s 5ms/step - loss: 0.0253 - acc: 0.2349 - val_loss: 0.0162 - val_acc: 0.2314
Epoch 2/100
625/625 [==============================] - 3s 5ms/step - loss: 0.0158 - acc: 0.2382 - val_loss: 0.0154 - val_acc: 0.2588
Epoch 3/100
625/625 [==============================] - 4s 7ms/step - loss: 0.0149 - acc: 0.2853 - val_loss: 0.0145 - val_acc: 0.3156
Epoch 4/100
625/625 [==============================] - 5s 8ms/step - loss: 0.0143 - acc: 0.3329 - val_loss: 0.0141 - val_acc: 0.3347
Epoch 5/100
625/625 [==============================] - 4s 7ms/step - loss: 0.0139 - acc: 0.3422 - val_loss: 0.0138 - val_acc: 0.3445
Epoch 6/100
625/625 [==============================] - 3s 5ms/step - loss: 0.0134 - acc: 0.3677 - val_loss: 0.0131 - val_acc: 0.3674
Epoch 7/100
625/625 [==============================] - 3s 5ms/step - loss: 0.0129 - acc: 0.3684 - val_loss: 0.0128 - val_acc: 0.3631
Epoch 8/100
625/625 [==============================] - 4s 7ms/step -

In [ ]:
X_test_transformed = pipeline.transform(X_test)

reconstructions = autoencoder.predict(X_test_transformed)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but Normalizer was fitted with feature names
  warnings.warn(


2651/2651 [==============================] - 6s 2ms/step


In [ ]:
import numpy as np
mse = np.mean(np.power(X_test_transformed - reconstructions, 2), axis=1)

In [ ]:
THRESHOLD = 3

def mad_score(points):
    """https://www.itl.nist.gov/div898/handbook/eda/section3/eda35h.htm """
    m = np.median(points)
    ad = np.abs(points - m)
    mad = np.median(ad)

    return 0.6745 * ad / mad

z_scores = mad_score(mse)
outliers = z_scores > THRESHOLD

In [ ]:
print(f"Detected {np.sum(outliers):,} anamolies in a total of {np.size(z_scores):,} transactions [{np.sum(outliers)/np.size(z_scores):.2%}].")

Detected 2,306 anamolies in a total of 84,807 transactions [2.72%].


In [ ]:
preds = []
for i in range(len(mse)):
  if mse[i]>0.0259:
    preds.append(1)
  else:
    preds.append(0)

In [ ]:
tp = 0
fp = 0
fn = 0
tn = 0

for i in range(len(preds)):
  if y_test[i]==0 and preds[i]==0:
    tn+=1
  if y_test[i]==1 and preds[i]==1:
    tp+=1
  if y_test[i]==0 and preds[i]==1:
    fp+=1
  if y_test[i]==1 and preds[i]==0:
    fn+=1

pr = tp / (tp + fp)
re = tp/ (tp + fn)

print("True Positives: ", tp)
print("False Positives: ", fp)
print("False Negatives: ", fn)
print("Precision: ", pr)
print("Recall: ", re)
print("F1 Score: ", (2*pr*re)/(pr+re))

True Positives:  327
False Positives:  388
False Negatives:  165
Precision:  0.45734265734265733
Recall:  0.6646341463414634
F1 Score:  0.5418392709196355
